# ECON 140 - Problem Set 3

## INSTRUCTIONS

* Please step through this problem set, produce or fix the code, and run it to see its output.

## Manipulate `data.frame` using `base R`

While packages like `tidyverse` and `dplyr` offer very neat ways to manipulate data, they rely on syntax quite different from `base R` and use namespaces that overlap with common `R` commands such that code written with these packages may suffer from a lack of portability and interfere with other R functions. In this exercise, we will look at some of the ways we can perform the same data manipulation techniques using `base R` just as efficiently and parsimoniously.

To get started, copy and paste the following code into the code block below and run it:
```R
packlist = c("haven")
install.packages(packlist[!(packlist %in% installed.packages()[, 1])])
wateruse <- haven::read_dta("wateruse.dta")
```

The dataset we just loaded into `R` is water use data taken from prior work of the UC Berkeley EEP/IAS 118 team. Run the code below to get a sense of what's in it.

In [ ]:
head(wateruse)

### Creating and transforming variables

So far we have seen some simple variable transformations such as logging and polynomials. More complex operations using more than one existing column can just be easily done in `R`.

Now, we are going to make a new variable called `gpd` calculated based on existing data. `gpd` stands for "gallons per day" and will be calculated by converting `unit` measured in CCFs, or "100 cubic feet," into gallons then dividing by `num_days`. The conversion factor from CCFs to gallons is 748.

Copy and paste the following code into the code block below and run it:
```R
wateruse$gpd <- wateruse$unit * 748 / wateruse$num_days
```

As you can see, if we need to use a lot of variables, needing to keep typing `wateruse$` can take up a lot of space. To pare it down, you can use the `with()` function to achieve the same result:
```R
wateruse$gpd <- with(wateruse, unit * 748 / num_days)
```
Copy and paste the following code into the code block below and run it to check if the two snippets produce the same results. You should get an output of 0.
```R
sum(wateruse$unit * 748 / wateruse$num_days - with(wateruse, unit * 748 / num_days))
```

Complete the following code to create a new variable `vpsf`, or "value per square foot," calculated by deviding `homeval` by `homesize`:

In [ ]:
wateruse$vpsf <-
summary(wateruse$vpsf)

### Subsetting `data.frame`

One way to achieve subsetting `data.frame` in `R` is by using the indexing syntax, which is quite versatile. Using row and columb numbers, you can slice and dice a `data.frame` anyway you like. Run the following code in the code block below:

```R
wateruse_1 <- wateruse[3:12, c(4, 6:7)]
wateruse_1
rm(wateruse_1)
```

Complete the code below to split `wateruse` into two `data.frames`, where `wateruse_a` contains rows 1 through 1000 and `wateruse_b` contains the rest.

In [ ]:
wateruse_a <- 
wateruse_b <- 
summary(wateruse_a)
summary(wateruse_b)
rm(wateruse_a)
rm(wateruse_b)

If you want to create a new `data.frame` that only contains the `unit`, `num_days`, and `zip` columns, you can do
```R
wateruse_2 <- wateruse[, c('unit', 'num_days', 'zip')]
head(wateruse_2)
rm(wateruse_2)
```
Run the code above in the code block below:

Conversely, if you want to remove columbs by name, you can do
```R
wateruse_3 <- wateruse[, !(names(wateruse) %in% c('unit', 'num_days', 'zip'))]
head(wateruse_3)
rm(wateruse_3)
```
Run the code above in the code block below:

As you can see, in the backend, `R` is using Boolean logic to select columns to keep and discard. The same idea applies to subsetting rows, which is also sometimes called "filtering," where you select only observations that satisfy certain criteria. For example, if you only want to keep observations that have 4 bathrooms or fewer:
```R
wateruse_4 <- wateruse[wateruse$num_baths <= 4, ]
summary(wateruse_4$num_baths)
rm(wateruse_4)
```
Run the code above in the code block below:

Using both row and colum indices, you can select only certain columns while filtering on rows. For example, if you want a `data.frame` with only `homeval` and `lotsize` for homes built after 1950, you can do
```R
wateruse_5 <- wateruse[wateruse$yearbuilt >= 1950, c('homeval', 'lotsize')]
summary(wateruse_5)
rm(wateruse_5)
```
Run the code above in the code block below:

Alternatively, you can use the `subset` function to achieve the same effect:
```R
wateruse_6 <- subset(wateruse, yearbuilt >= 1950, c('homeval', 'lotsize'))
summary(wateruse_6)
rm(wateruse_6)
```

Use any method, complete and run the following code to create a subset of `wateruse` called `wateruse_7` that contains all columns except the first three for only rows where `num_beds` is less than 5 and `yearbuilt` is after 1950.

In [ ]:
wateruse_7 <-
summary(wateruse_7)
rm(wateruse_7)
invisible(gc())

### Dealing with missing data

Regression functions in more recent versions of `R` (meaning within the past few years) can typically deal with missing data can automatically omitting rows that contain them. Care must be given in just getting rid of data with missing variables, as this may introduce bias into analysis down the line (for example, a certain type of person may be more likely to skip a specific question on a survey). Sometimes, manual cleanup may be required.

To get rid of ALL rows with ANY missing data, you can use:
```R
wateruse_no.na <- na.omit(wateruse)
dim(wateruse)
dim(wateruse_no.na)
```
Run the code above in the code block below:

This is obvious not desirable for the aforementioned reasons unless there's a specific justification. Run the following code to see how many missing values exist for each column:

In [ ]:
colSums(is.na(wateruse))

where `is.na()` returns a Boolean for each cell of the `data.frame` whether it's missing data and `colSums()` sums each column. So when you're running into problems with, say

In [ ]:
summary(lm(homeval ~ yearbuilt + homesize + poly(num_baths, 2),
        data = wateruse))

You can just remove the rows missing data for `num_baths` only by subsetting

In [ ]:
wateruse_8 <- wateruse[!(is.na(wateruse$num_baths)), ]
summary(lm(homeval ~ yearbuilt + homesize + poly(num_baths, 2),
        data = wateruse_8))
rm(wateruse_8)

### Aggregating and merging data

Aggregation in `base R` can be done with the `aggregate()` function, which is not the most straightforward command, but a very useful one nonetheless. For example, to get the sum of all `unit` by `billingcycle`, we can do:
```R
agg_unit_by_bc <- aggregate(list(unit_sum = wateruse$unit),
                            wateruse[, c("billingcycle")],
                            FUN = sum)
summary(agg_unit_by_bc)
```
Run the code above in the code block below:

The `list()` construction in the first argument is optional, although using `list()` allows us to name the aggregated resulting column in one fell swoop. The `list()` construction for the second argument, or the categories to aggregate by, is mandatory, although `data.frame` is a special type of list already, and coding it this way preserves the column names for `billingcycle` without having to respecify it. The third argument specifies the kind of operation to run for the aggregation, in this case, a sum.

We can just as easily aggregate by more than one set of categories. For example, if we want total units per billing cycle per city, *i.e.* sums for each unique pairwise combination of billing cycles and cities, we can do:
```R
agg_unit_by_bc.x.city <- aggregate(list(unit_sum = wateruse$unit),
                            wateruse[, c("billingcycle", "city")],
                            FUN = sum)
summary(agg_unit_by_bc.x.city)
```
Run the code above in the code block below:

A lot of times, we need to combine data from difference sources, often of differing resolutions. For example, we may have one set of data of county-level statistics for all of US and another set with state-level statistics, and we need to combine them such that each county row would also have a matching state-level value. Other times, we may have wages for each employee from one table, and title from another, and we need to combine them into the same dataset. We can use the `merge()` function to stitch together these datasets.

Aggregated values are sometimes passed back down to observation level. The following code would merge `agg_unit_by_bc.x.city` and `wateruse` to replace `wateruse` with a new dataset with one additional column called `unit_sum` that matches the per `billingcycle` per `city` sum of `unit`s to each observation in that city and billing cycle.
```R
wateruse <- merge(wateruse, agg_unit_by_bc.x.city)
head(wateruse, 3)
```
Run the code above in the code block below:

Pretty easy, right? As long as the categories by which two datasets are merged by have identical column names, and no other columns from the two datasets share the same names, this construction will do. Otherwise, you will need to specify the column names to merge by with another argument `by = c(...)`. Additional arguments provides for whether to keep unmatched rows from either dataset.

Complete the following code to assign a random number to each unique zip code from `wateruse` in a new `data.frame` called `zip_rand`, then merge the two datasets by zip code to replace the original `wateruse`.

In [ ]:
zip <- unique(wateruse$zip)
rand <- rnorm(length(zip))
zip_rand <- as.data.frame(list(zip = zip, rand = rand))
wateruse <- merge()
head(wateruse, 3)

### Resampling

Resampling refers to creating new samples based on a fixed sample. It is useful for a range of statistical, econometric and data scientific application, such as resampling-based estimation methods and cross-validation. Resampling can be done randomly or non-randomly. A non-randomly resampling example is simply a subset that omits a certain number of observations. Random resampling can be done with or without replacement, each with different properties and uses.

The simplest way to resample from a fixed sample is to use the `sample` function. To create a random resample of 1000 observations from `wateruse` without replacement (which is by default), run the following code:

In [ ]:
wateruse_9 <- wateruse[sample(1:dim(wateruse)[1], 1000), ]
summary(wateruse_9)
rm(wateruse_9)

To create a random resample of `wateruse` with replacement, add flag `replace = TRUE`. This is like drawing from a deck of cards, then returning the drawn card to the deck and shuffle before the next draw. You can therefore resample at a sample size larger than the original sample size. Run the following code for an example:

In [ ]:
wateruse_10 <- wateruse[sample(1:dim(wateruse)[1], 10000, replace = TRUE), ]
summary(wateruse_10)
rm(wateruse_10)
invisible(gc())

The downside of `sample()` is that we don't easily get what's leftout. One way to get around this is to sample from just a list of row numbers without replacement, and randomly split the dataset by using those row numbers as indices. Complete the following code to perform such a task:

In [ ]:
sample_size <- dim(wateruse)[1]
split_a <- sample(1:sample_size, 1000)
split_b <- (1:sample_size)[!(1:sample_size %in% split_a)]
wateruse_a <-
wateruse_b <-
summary(wateruse_a)
summary(wateruse_b)

## Fitting values and making predictions

Although fitting models to make good predictions isn't the focus of this course, it is nonetheless a central use case of econometrics and data science, and we should know at least how to make one, good or bad, based on a regression result. Calculating fitted values and making predictions are identical processes, the only difference is that fitted values are made *in-sample* whereas predictions are made *out-of-sample*. 

For simplicity, we are going to get rid of all missing values from the previously produced `wateruse_a` and `wateruse_b` `data.frame` objects (even though we should be careful about doing so in a real setting). These will serve as our in-sample, or *training set* and out-of-sample, or *validation set*, so to speak, though keep in mind we won't be doing any real cross validation or optimization, which are outside the scope of this course. Run the following code:


In [ ]:
wateruse_a <- na.omit(wateruse_a)
wateruse_b <- na.omit(wateruse_b)


### Fitted values

Consider the following regression:

In [ ]:
lm1 <- lm(homeval ~ homesize + num_baths + num_beds + num_rooms + lotsize,
            data = wateruse_a)

We already know that the fitted values are stored within `lm1$fitted.values`, which can also be accessed using the command `fitted.values(lm1)`. We can also manually calculate the fitted values by calculating the linear combination of estimated parameters and the covariates:
```R
wateruse_a$fitted_homeval_auto <- lm1$fitted.values
wateruse_a$fitted_homeval_manual <- as.matrix(cbind(1, wateruse_a[, c("homesize",
                                                    "num_baths",
                                                    "num_beds",
                                                    "num_rooms",
                                                    "lotsize")])) %*% 
                                                    lm1$coefficients
sum(wateruse_a$fitted_homeval_auto - wateruse_a$fitted_homeval_manual)
```
Run the code above in the code block below:

The code above should produce an output of 0, or close to 0 if there's rounding differences, if the manual and auto fitted values are the same. The manual method is also how you would make predictions using the out of sample data:
```R
wateruse_b$predicted_homeval_manual <- cbind(1, as.matrix(wateruse_b[, c("homesize",
                                                    "num_baths",
                                                    "num_beds",
                                                    "num_rooms",
                                                    "lotsize")])) %*% 
                                                    lm1$coefficients
```
Run the code above in the code block below:

Note the small changes in the code, switching from training set to validation set. However, this method is way too cumbersome. It's much easier just to use the `predict()` function:
```R
wateruse_b$predicted_homeval_easy <- predict(lm1, wateruse_b)
sum(wateruse_b$predicted_homeval_easy - wateruse_b$predicted_homeval_manual)
```

The code above should produce an output of 0, or close to 0 if there's rounding differences, if the manual and easy predicted values are the same. The `predict()` function works by transferring the parameter estimates from the regression results onto the new dataset as long as the new dataset has the same covariates that are used in the regression, even if you use in-formula transformations like `poly()`, `factor()`, `log()`, and interactions.

In this simple example, lets compare the root mean squared error [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation), a measure of prediction accuracy, of fitted values and predicted values:
```R
rmse <- function(observed, predicted) sqrt(sum((observed - predicted)^2)/length(observed))
rmse_fitted <- rmse(wateruse_a$homeval, lm1$fitted.values)
rmse_predicted <- rmse(wateruse_b$homeval, predict(lm1, wateruse_b))
cbind(rmse_fitted, rmse_predicted)
```
Run the code above in the code block below:

Smaller RMSE means higher accuracy. Does this result surprise you?

For your last exercise, produce the RMSE for the in-sample fitted values and out-of-sample predicted values by completing the following code, then write down in a couple sentences what you think of the result:

In [ ]:
lm2 <- lm(homeval ~ poly(yearbuilt, 2) + poly(homesize, 2) + 
            poly(num_baths, 2) + poly(num_beds, 2) + poly(num_rooms, 2) + 
            poly(lotsize, 2) + log(homesize * yearbuilt), data = wateruse_a)
rmse_fitted_2 <- 
rmse_predicted_2 <-
cbind(rmse_fitted_2, rmse_predicted_2)

Write your thoughts here: 